# Problem Statement and Description

In the Union Territory of New Delhi, there are a lot of places for the residents to go out and eat, and there are people who would like to take the opportunity of serving such people by opening his/her own restraunt. Food business always thrives on the quality of the food served, variety of the dishes and area in which it is operting. But, in large cities like New Delhi, it could be difficult to decide where should one open his new restraunt, after which he would have to decide which cuisines to offer and many more things. No worries, these problems could be easily solved by using the data, and applying certain tools on it.

So, the Final Problem is to-

1. Classify different areas in different Clusters
2. Decide where to open a restraunt

# Data Requirements

Since we have established a problem, we have to go ahead and decide what kinds of data would be needed. To solve the problem for deciding places where people usually go to eat, we have to determine that by clustering the neighborhoods into diiferent categories, and then determine which could be a more appropriate place to operate at. Then we would need the 

Important links:

1. https://simple.wikipedia.org/wiki/List_of_districts_in_Delhi

These are the sources used in identifying areas in Delhi for further segmentation, analysis and finding optimum place for a person to open a restaraunt.

Apart form this data, Foursquare API will be used to gather data regarding the places in each neighborhood, so that a more detailed and thorough analysis could be conducted by clustering different areas according to the types of places present in the locality and proper findings can be shown.

In [122]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

#### Load and explore the data

In [113]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup as bs
from sklearn.cluster import KMeans
import sys
import folium 

print('Libraries imported.')

Libraries imported.


In [4]:
url="https://simple.wikipedia.org/wiki/List_of_districts_in_Delhi"
req=requests.get(url)

In [8]:
soup = bs(req.content,'lxml')
table = soup.find_all('table')

In [61]:
res={'District':[],'Sub Division':[]}
keys=list(res.keys())
pos=0
for item in table[0].find_all('td'):
    if pos==0:
        sno=item.text.replace('\n',"")
    elif pos==1:
        dis=item.text.replace('\n',"")
    else:
        if "[" in item.text:
            sd=item.text.replace("\n","")[:item.text.find("[")]
        else:
            sd=item.text.replace("\n","")
        res[keys[0]].append(dis)
        res[keys[1]].append(sd)
    pos+=1
    if pos==6:
        pos=0

In [104]:
neigh=pd.DataFrame(res).drop_duplicates('Sub Division').reset_index(drop=True)
neigh

,District,Sub Division
0,New Delhi,Connaught Place
1,New Delhi,Chanakyapuri
2,New Delhi,Delhi Cantonment
3,New Delhi,Vasant Vihar
4,North West Delhi,Narela
5,North West Delhi,Model Town
6,North West Delhi,Alipur
7,North Delhi,Kanjhawala
8,North Delhi,Rohini
9,North Delhi,Saraswati Vihar


These are the different neighborhoods in New Delhi. Now we will Geolocator to for adding Location details to each neighborhood for usage in Foursquare API

In [ ]:
res_add={'Latitude':[],'Longitude':[]}
geolocator = Nominatim(user_agent="dl_explorer")
for sub,dis in zip(neigh["Sub Division"],neigh["District"]):
    add=sub+" , "+dis+" , "+ " , Delhi"
    location=""
    while location=="":        
        try: location = geolocator.geocode(add)
            
        except: continue
    
    lat = location.latitude
    lng = location.longitude
    res_add['Latitude'].append(lat)
    res_add['Longitude'].append(lng)

In [115]:
neighborhoods=pd.concat((neigh,pd.DataFrame(res_add)),axis=1)
neighborhoods

,District,Sub Division,Latitude,Longitude
0,New Delhi,Connaught Place,28.631383,77.219792
1,New Delhi,Chanakyapuri,28.594677,77.188521
2,New Delhi,Delhi Cantonment,28.593833,77.134979
3,New Delhi,Vasant Vihar,28.560691,77.160791
4,North West Delhi,Narela,28.842610,77.091835
5,North West Delhi,Model Town,28.702714,77.193991
6,North West Delhi,Alipur,28.795996,77.136071
7,North Delhi,Kanjhawala,28.735817,77.004315
8,North Delhi,Rohini,28.716209,77.117074
9,North Delhi,Saraswati Vihar,28.700224,77.124936


These are the localities that will be used in the API for futher data collection.

In [191]:
address = 'Delhi'

geolocator = Nominatim(user_agent="dl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Delhi are 28.6517178, 77.2219388.


In [204]:
LIMIT=1000
radius=100000
url=url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude,
    longitude,
    radius, 
    LIMIT)

In [205]:
req=requests.get(url).json()

In [117]:
Delhi_Map=folium.Map(location=[latitude,longitude],zoom_start=10)
for sub,lat,lng in zip(neighborhoods['Sub Division'],neighborhoods['Latitude'],neighborhoods['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=sub,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Delhi_Map)
Delhi_Map